In [1]:
conda install -c conda-forge librosa

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - librosa


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    audioread-2.1.8            |           py37_1          32 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    cffi-1.13.2                |   py37h8022711_0         220 KB  conda-forge
    ffmpeg-4.1.3               |       h167e202_0        75.8 MB  conda-forge
    gnutls-3.6.5               |    hd3a4fd2_1002         2.1 MB  conda-forge
    lame-3.100                 |    h14c3975_1001         498 KB  co

In [1]:
import IPython
from scipy.io import wavfile
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy.signal import butter, lfilter
%matplotlib inline


In [2]:
def S1S2(x):
    x=x/max(np.abs(x))
    s=-1*(x*x)*np.log(x*x)
    a=np.ones(201)
    ss=np.convolve(s,a)
    ss=ss/np.max(ss)
    x2=x*(ss>0.3)[:len(x)]
    x2=x2+0.0000000000000000001
    senv=-1*(x2*x2)*np.log(x2*x2)
    a=np.ones(201)
    senv=np.convolve(senv,a)
    senv=senv/np.max(ss)
    sp=scipy.signal.hilbert(senv)
    ph=np.abs(np.angle(senv+(1j)*sp))
    der=np.zeros(len(ph))
    start=[]
    enda=[]
    flag=0
    for i in range(len(senv)-1):
        der[i]=ph[i+1]-ph[i]
        if der[i]>0 and  ph[i]>0.1 and flag == 0  :
            start.append(i)
            flag=1
        if der[i]<-2.0 :
            flag=0
    flag=0        
    for i in range(len(senv)-1):
        if der[i]>0 and  ph[i]>3.0 and flag == 0  :
            enda.append(i)
            flag=1
        if der[i]<-2.0 :
            flag=0
    return (start,enda)


In [ ]:
def S1_S2_classifier(signal, start_index, end_index, sampling_rate):              

    I = []#np.array() # kth element will tell the length of the kth interval
    D = []#np.array() # kth element will tell the duration of lub/dub in the kth interval
    A = []#np.array() # sum of all values of the signal(abs) in the kth duration
    for x in range (0, len(start_index)-1):
        i_kth = start_index[x+1] - start_index[x]
        I.append(i_kth)
    #check for first element in end_index
    if end_index[0] < start_index[0]:
        np.delete(end_index, 0)

    for x in range (0, len(end_index)-1):
        d_kth = end_index[x] - start_index[x]
        D.append(d_kth)
    sig = np.absolute(signal)
    for x in range (0, len(start_index)-1):
        sum = 0;
        for y in range (start_index[x],start_index[x] + D[x]):
            sum += sig[y]
        A.append(sum)
   
    ### All the arrays I, D and A have been constructed ###
    label = []#np.array()
    for k in range (0, len(start_index)-2):
        if I[k] < I[k+1]:
            if (I[k+1]-I[k] > (0.05*sampling_rate)):
                label.append(1)
            elif (D[k] > D[k+1]) and (D[k]-D[k+1]) > 0.005*sampling_rate:
                label.append(1)
            elif (A[k] > 1.5*A[k+1]):
                label.append(1)
            else:
                label.append(2)
        if I[k] > I[k+1]:
            if (I[k]-I[k+1] > (0.05*sampling_rate)):
                label.append(2)
            elif (D[k] < D[k+1]) and (D[k+1]-D[k]) > 0.005*sampling_rate:
                label.append(2)
            elif (A[k] < 1.5*A[k+1]):
                label.append(2)
            else:
                label.append(1)
        
   
    return label


In [3]:
wav_loc = "abc.wav"
src_rate, src_data = wavfile.read(wav_loc)
src_data = src_data[28000:]#/3276
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 4))
plt.plot(src_data, color="black")
IPython.display.Audio(data=src_data, rate=src_rate)

FileNotFoundError: [Errno 2] No such file or directory: 'abc.wav'

In [4]:
start,enda=S1S2(src_data)

NameError: name 'src_data' is not defined

In [ ]:
labels=S1_S2_classifier(src_data,start,enda,src_rate)

In [ ]:
labels